# 📊 Création Automatique des Mesures DAX - Marketing 360

Ce notebook crée automatiquement toutes les mesures DAX dans le Semantic Model Fabric.

## Prérequis :
- Semantic Model créé dans Fabric
- XMLA endpoint activé sur le workspace
- Package Python : `semantic-link-labs` installé

## Mesures créées :
- **CRM** : 9 mesures (customers, CLV, NPS, churn, etc.)
- **Marketing** : 12 mesures (campaigns, email rates, ROI, etc.)
- **Commerce** : 6 mesures (orders, revenue, returns, etc.)
- **Attribution** : 3 mesures (marketing attribution)
- **Customer 360** : 4 mesures (combinées)
- **Temporelles** : 2 mesures (YTD, MTD)

**Total : 36 mesures DAX**

In [ ]:
# Cell 1: Installation des packages (si nécessaire)
print("📦 Installation des packages nécessaires...\n")

# Décommenter si vous devez installer les packages
# %pip install semantic-link-labs
# %pip install azure-identity

print("✅ Packages prêts")

In [ ]:
# Cell 2: Imports
import sempy
import sempy.fabric as fabric
from sempy.fabric import FabricDataFrame
import pandas as pd

print("✅ Imports terminés")

In [ ]:
# Cell 3: Configuration - À MODIFIER SELON VOTRE ENVIRONNEMENT

# Nom de votre workspace Fabric
WORKSPACE_NAME = "Demo-Marketing360"  # ⚠️ MODIFIER ICI

# Nom de votre Semantic Model
DATASET_NAME = "Marketing360_SemanticModel"  # ⚠️ MODIFIER ICI

print(f"📊 Configuration:")
print(f"   Workspace: {WORKSPACE_NAME}")
print(f"   Dataset: {DATASET_NAME}")

In [ ]:
# Cell 4: Définition de toutes les mesures DAX

dax_measures = [
    # ============================================
    # Mesures CRM
    # ============================================
    {
        "name": "Total Customers",
        "expression": "COUNTROWS(crm_customers)",
        "table": "crm_customers",
        "format": "#,##0"
    },
    {
        "name": "Active Customers",
        "expression": "CALCULATE([Total Customers], crm_customers[status] = \"active\")",
        "table": "crm_customers",
        "format": "#,##0"
    },
    {
        "name": "Churned Customers",
        "expression": "CALCULATE([Total Customers], crm_customers[status] = \"churned\")",
        "table": "crm_customers",
        "format": "#,##0"
    },
    {
        "name": "Churn Rate %",
        "expression": "DIVIDE([Churned Customers], [Total Customers], 0) * 100",
        "table": "crm_customers",
        "format": "0.00%"
    },
    {
        "name": "Avg CLV",
        "expression": "AVERAGE(crm_customer_profile[clv_score])",
        "table": "crm_customer_profile",
        "format": "#,##0"
    },
    {
        "name": "Avg Churn Risk",
        "expression": "AVERAGE(crm_customer_profile[churn_risk_score])",
        "table": "crm_customer_profile",
        "format": "#,##0"
    },
    {
        "name": "Avg NPS",
        "expression": "AVERAGE(crm_customer_profile[nps_last])",
        "table": "crm_customer_profile",
        "format": "0.0"
    },
    {
        "name": "Total Interactions",
        "expression": "COUNTROWS(crm_interactions)",
        "table": "crm_interactions",
        "format": "#,##0"
    },
    {
        "name": "Avg Satisfaction",
        "expression": "AVERAGE(crm_interactions[satisfaction_score])",
        "table": "crm_interactions",
        "format": "0.0"
    },
    
    # ============================================
    # Mesures Marketing
    # ============================================
    {
        "name": "Total Campaigns",
        "expression": "COUNTROWS(marketing_campaigns)",
        "table": "marketing_campaigns",
        "format": "#,##0"
    },
    {
        "name": "Active Campaigns",
        "expression": "CALCULATE([Total Campaigns], marketing_campaigns[status] = \"active\")",
        "table": "marketing_campaigns",
        "format": "#,##0"
    },
    {
        "name": "Total Marketing Budget",
        "expression": "SUM(marketing_campaigns[budget_eur])",
        "table": "marketing_campaigns",
        "format": "€#,##0"
    },
    {
        "name": "Total Email Sends",
        "expression": "COUNTROWS(marketing_sends)",
        "table": "marketing_sends",
        "format": "#,##0"
    },
    {
        "name": "Total Email Events",
        "expression": "COUNTROWS(marketing_events)",
        "table": "marketing_events",
        "format": "#,##0"
    },
    {
        "name": "Email Opens",
        "expression": "CALCULATE([Total Email Events], marketing_events[event_type] = \"open\")",
        "table": "marketing_events",
        "format": "#,##0"
    },
    {
        "name": "Open Rate %",
        "expression": "DIVIDE([Email Opens], [Total Email Sends], 0) * 100",
        "table": "marketing_events",
        "format": "0.00%"
    },
    {
        "name": "Email Clicks",
        "expression": "CALCULATE([Total Email Events], marketing_events[event_type] = \"click\")",
        "table": "marketing_events",
        "format": "#,##0"
    },
    {
        "name": "Click Rate %",
        "expression": "DIVIDE([Email Clicks], [Total Email Sends], 0) * 100",
        "table": "marketing_events",
        "format": "0.00%"
    },
    {
        "name": "Email Bounces",
        "expression": "CALCULATE([Total Email Events], marketing_events[event_type] = \"bounce\")",
        "table": "marketing_events",
        "format": "#,##0"
    },
    {
        "name": "Bounce Rate %",
        "expression": "DIVIDE([Email Bounces], [Total Email Sends], 0) * 100",
        "table": "marketing_events",
        "format": "0.00%"
    },
    {
        "name": "Email Unsubscribes",
        "expression": "CALCULATE([Total Email Events], marketing_events[event_type] = \"unsubscribe\")",
        "table": "marketing_events",
        "format": "#,##0"
    },
    {
        "name": "Unsubscribe Rate %",
        "expression": "DIVIDE([Email Unsubscribes], [Total Email Sends], 0) * 100",
        "table": "marketing_events",
        "format": "0.00%"
    },
    
    # ============================================
    # Mesures Commerce
    # ============================================
    {
        "name": "Total Orders",
        "expression": "COUNTROWS(orders)",
        "table": "orders",
        "format": "#,##0"
    },
    {
        "name": "Total Revenue",
        "expression": "SUM(orders[total_amount_eur])",
        "table": "orders",
        "format": "€#,##0"
    },
    {
        "name": "Avg Order Value",
        "expression": "DIVIDE([Total Revenue], [Total Orders])",
        "table": "orders",
        "format": "€#,##0.00"
    },
    {
        "name": "Total Returns",
        "expression": "COUNTROWS(returns)",
        "table": "returns",
        "format": "#,##0"
    },
    {
        "name": "Return Rate %",
        "expression": "DIVIDE([Total Returns], [Total Orders], 0) * 100",
        "table": "returns",
        "format": "0.00%"
    },
    {
        "name": "Total Products Sold",
        "expression": "SUM(order_lines[qty])",
        "table": "order_lines",
        "format": "#,##0"
    },
    
    # ============================================
    # Mesures d'Attribution Marketing
    # ============================================
    {
        "name": "Marketing Attributed Orders",
        "expression": "CALCULATE([Total Orders], NOT(ISBLANK(orders[attributed_campaign_id])))",
        "table": "orders",
        "format": "#,##0"
    },
    {
        "name": "Marketing Attributed Revenue",
        "expression": "CALCULATE([Total Revenue], NOT(ISBLANK(orders[attributed_campaign_id])))",
        "table": "orders",
        "format": "€#,##0"
    },
    {
        "name": "Marketing ROI %",
        "expression": "DIVIDE([Marketing Attributed Revenue] - [Total Marketing Budget], [Total Marketing Budget], 0) * 100",
        "table": "orders",
        "format": "0.00%"
    },
    
    # ============================================
    # Mesures Combinées Customer 360
    # ============================================
    {
        "name": "Revenue per Customer",
        "expression": "DIVIDE([Total Revenue], [Total Customers], 0)",
        "table": "orders",
        "format": "€#,##0.00"
    },
    {
        "name": "Orders per Customer",
        "expression": "DIVIDE([Total Orders], [Total Customers], 0)",
        "table": "orders",
        "format": "0.00"
    },
    {
        "name": "Customers Who Ordered",
        "expression": "CALCULATE(DISTINCTCOUNT(orders[customer_id]))",
        "table": "orders",
        "format": "#,##0"
    },
    {
        "name": "Conversion Rate %",
        "expression": "DIVIDE([Customers Who Ordered], [Total Customers], 0) * 100",
        "table": "orders",
        "format": "0.00%"
    },
    
    # ============================================
    # Mesures Temporelles
    # ============================================
    {
        "name": "Revenue YTD",
        "expression": "TOTALYTD([Total Revenue], orders[order_at])",
        "table": "orders",
        "format": "€#,##0"
    },
    {
        "name": "Orders MTD",
        "expression": "TOTALMTD([Total Orders], orders[order_at])",
        "table": "orders",
        "format": "#,##0"
    }
]

print(f"📋 Nombre total de mesures à créer: {len(dax_measures)}")
print(f"\n📊 Répartition par catégorie:")
print(f"   - CRM: 9 mesures")
print(f"   - Marketing: 13 mesures")
print(f"   - Commerce: 6 mesures")
print(f"   - Attribution: 3 mesures")
print(f"   - Customer 360: 4 mesures")
print(f"   - Temporelles: 2 mesures")

In [ ]:
# Cell 5: Fonction pour créer les mesures

def create_dax_measures(workspace, dataset, measures_list):
    """
    Crée les mesures DAX dans le Semantic Model Fabric
    
    Args:
        workspace: Nom du workspace
        dataset: Nom du semantic model
        measures_list: Liste des mesures à créer
    """
    success_count = 0
    error_count = 0
    errors = []
    
    print(f"🚀 Début de la création des mesures...\n")
    
    for i, measure in enumerate(measures_list, 1):
        try:
            # Créer la mesure
            fabric.create_measure(
                dataset=dataset,
                workspace=workspace,
                measure_name=measure['name'],
                expression=measure['expression'],
                table_name=measure['table'],
                format_string=measure.get('format', '')
            )
            
            print(f"  ✅ [{i}/{len(measures_list)}] {measure['name']}")
            success_count += 1
            
        except Exception as e:
            print(f"  ❌ [{i}/{len(measures_list)}] {measure['name']}: {str(e)}")
            error_count += 1
            errors.append({"measure": measure['name'], "error": str(e)})
    
    # Résumé
    print(f"\n" + "="*60)
    print(f"📊 RÉSUMÉ")
    print(f"="*60)
    print(f"✅ Mesures créées avec succès: {success_count}")
    print(f"❌ Erreurs: {error_count}")
    
    if errors:
        print(f"\n⚠️ Détails des erreurs:")
        for err in errors:
            print(f"  - {err['measure']}: {err['error']}")
    
    return success_count, error_count, errors

print("✅ Fonction create_dax_measures définie")

In [ ]:
# Cell 6: Exécuter la création des mesures

print("🎯 Création des mesures DAX dans le Semantic Model...\n")

success, errors, error_details = create_dax_measures(
    workspace=WORKSPACE_NAME,
    dataset=DATASET_NAME,
    measures_list=dax_measures
)

print(f"\n🎉 Processus terminé!")
print(f"\n📌 Prochaines étapes:")
print(f"   1. Ouvrir le Semantic Model dans Fabric")
print(f"   2. Vérifier que les mesures apparaissent dans les tables")
print(f"   3. Tester les mesures dans un rapport Power BI")

In [ ]:
# Cell 7: (Optionnel) Lister les mesures existantes

print("📋 Liste des mesures dans le Semantic Model:\n")

try:
    # Lister toutes les mesures existantes
    measures = fabric.list_measures(
        dataset=DATASET_NAME,
        workspace=WORKSPACE_NAME
    )
    
    if measures:
        print(f"Total: {len(measures)} mesures\n")
        for measure in measures:
            print(f"  • {measure['Name']} (Table: {measure['Table']})")
    else:
        print("Aucune mesure trouvée")
        
except Exception as e:
    print(f"⚠️ Impossible de lister les mesures: {str(e)}")
    print("Vérifiez que le workspace et le dataset sont corrects")